## 1 - Imports

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.chdir("..")

In [3]:
%pip install tensorflow --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path

from src.utils.best_model_utils import load_best_table, load_best_model
from src.models.dl_utils import create_sequences, scale_data
from src.models.evaluation import evaluate_regression


## 2 - Configuration

In [6]:
import yaml

def load_config(config_path: str = "config.yaml") -> dict:
    """
    Lit le fichier YAML de configuration et renvoie un dict.
    """
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg

cfg = load_config("config.yaml")
proc_dir    = Path(cfg["paths"]["data"]["processed"])
reports_dir = Path("outputs/reports")

# Charger la table des meilleurs modèles
best_df = load_best_table()
zones   = best_df['zone'].tolist()
horizons = ["daily", "hourly"]


## 3 - Widgets de sélection

In [7]:
zone_widget    = widgets.Dropdown(options=zones, description="Zone")
horizon_widget = widgets.Dropdown(options=horizons, description="Horizon")
ui = widgets.HBox([zone_widget, horizon_widget])
display(ui)


## 4 - Fonction de mise à jour

In [8]:
out = widgets.Output()

def update_dashboard(change):
    out.clear_output()
    zone    = zone_widget.value
    horizon = horizon_widget.value

    # 1. Chargement des données
    df = pd.read_parquet(proc_dir / f"{zone}_processed_{horizon}.parquet")
    split = int(len(df)*0.8)
    df_train, df_test = df.iloc[:split], df.iloc[split:]

    # 2. Affichage Stats
    with out:
        print(f"### Statistiques pour {zone} ({horizon})")
        display(df["demand"].describe())

        # 3. Charger et afficher le meilleur modèle
        model = load_best_model(zone, horizon)
        print(f"**Modèle sélectionné** : {type(model).__name__}")

        # 4. Préparation des données de test
        if hasattr(model, "predict") and model.__class__.__module__.startswith("tensorflow"):
            # Deep Learning
            LOOKBACK = 24 if horizon=="hourly" else 7
            features = [c for c in df.columns if c!="demand"]
            X_seq, y_seq = create_sequences(df, "demand", features, LOOKBACK)
            n_test = len(df_test)
            X_test_seq = X_seq[-n_test:]
            _, X_test_seq_scaled, _ = scale_data(X_seq[:-n_test], X_test_seq)
            y_true = y_seq[-n_test:].ravel()
            y_pred = model.predict(X_test_seq_scaled).ravel()
            idx = df_test.index
        else:
            # Sklearn / ARIMA
            X_test = df_test.drop(columns="demand")
            try:
                y_pred = model.predict(X_test)
            except:
                y_pred = model.predict(n_periods=len(df_test))
            y_true = df_test["demand"].values
            idx    = df_test.index

        # 5. Plot Réel vs Prédiction
        fig, ax = plt.subplots(figsize=(10,4))
        ax.plot(idx, y_true, label="Réel")
        ax.plot(idx, y_pred, label="Prédit", alpha=0.7)
        ax.set_title(f"Demande — {zone} ({horizon})")
        ax.legend()
        plt.show()

        # 6. KPI
        kpi = evaluate_regression(None, None, pd.Series(y_true), pd.Series(y_pred))
        print("**Performance**")
        print(f"MAE:  {kpi['MAE']:.2f}")
        print(f"RMSE: {kpi['RMSE']:.2f}")
        print(f"MAPE: {kpi['MAPE']:.2f}%")

widgets.interact(update_dashboard, change=zone_widget)  # lien zone
widgets.interact(update_dashboard, change=horizon_widget)  # lien horizon
display(out)


interactive(children=(Dropdown(description='Zone', index=1, options=('Peninsule_Iberique', 'Baleares', 'Canari…

interactive(children=(Dropdown(description='Horizon', options=('daily', 'hourly'), value='daily'), Output()), …

Output()

## 6 - Lancer l’application Streamlit

In [ ]:
import os
print("Démarrage du dashboard Streamlit… (Ctrl+C pour arrêter)")
os.system("streamlit run dashboard/app.py")


Démarrage du dashboard Streamlit… (Ctrl+C pour arrêter)
